In [1]:
import numpy as np
import pandas as pd
import re
from numpy.linalg import norm

In [2]:
with open('cats.txt','r') as file:
    cats = file.readlines()

sentences = pd.DataFrame(cats,columns=['sentences'])

In [3]:
# пробегаемся по каждому предложению и находим там только слова с помощю регулярок
cats = [re.findall('[a-z]+',cat.lower()) for cat in cats if cat]

# фунция для создания уникального "мешка слов"
def bag_of_words(sentences):
    temp = []
    for sentence in sentences:
        for word in sentence:
            if word not in temp:
                temp.append(word)
    return temp

# функция для оперделения косинусного расстояния двух векторов
def cosine_dist(pair):
    i,j = pair
    return (i@j).sum()/(norm(i)*norm(j))
bag = bag_of_words(cats)

# создаём столбцы для каждого слова в таблице и заполняем их нулями
for word in bag:
    sentences[word] = np.zeros((sentences.shape[0]))

# заполняем таблицу частотой слов в каждом предложении
for e,c in enumerate(cats):
    for word in c:
        word_column = sentences[word]
        word_column[e] += 1

# находим косинусное расстояние для каждого предложения с первым
results = []
for i in range(sentences.shape[0]):
    data = sentences.copy()
    true_cats = np.array(data.iloc[0,1:].to_list())
    comp_cats = np.array(data.iloc[i,1:].to_list())
    res = cosine_dist([true_cats,comp_cats])
    results.append(res)

d:\projects\pyinstagram\venv\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
# находим топ-3 близлежащих к первому предложению ответа
res = np.sort(results).tolist()
res.reverse()
top3 = res[1:4]
sentences['results'] = results

In [10]:
# выводим топ-3 предложения по ответам, найденным в предыдущей ячейке
for one in top3:
    idx = sentences.index[sentences['results']==one].to_list()
    print(idx)
    print(sentences['sentences'][idx[0]])

[6]
Domestic cats are similar in size to the other members of the genus Felis, typically weighing between 4 and 5 kg (8,8 and 11,0 lb).

[4]
In one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.

[21]
The fifth major update to Mac OS X, Leopard, contains such a mountain of features - more than 300 by Apple's count.


0     1.000000
1     0.047246
2     0.135526
3     0.104828
4     0.222911
5     0.059761
6     0.267261
7     0.074125
8     0.115728
9     0.094491
10    0.167183
11    0.119523
12    0.160357
13    0.129641
14    0.125988
15    0.055728
16    0.159364
17    0.043355
18    0.055728
19    0.111456
20    0.157243
21    0.174964
Name: results, dtype: float64